In [2]:
import geopandas as gpd
import requests
import json
from shapely.geometry import Polygon

# Example GeoDataFrame of school roofs
data = {
    'name': ['School A', 'School B'],
    'coordinates': [
        [[40.7128, -74.0060], [40.7129, -74.0061], [40.7129, -74.0059], [40.7128, -74.0058]],  # Polygon for School A
        [[34.0522, -118.2437], [34.0523, -118.2438], [34.0523, -118.2436], [34.0522, -118.2435]]  # Polygon for School B
    ],
    'elevation': [10, 20]  # Elevation for each roof
}

# Create a GeoDataFrame
roof_df = gpd.GeoDataFrame(data, geometry=[Polygon(coords) for coords in data['coordinates']], crs="EPSG:4326")

# Function to convert the GeoDataFrame to the format expected by the FastAPI endpoint
def geodataframe_to_json(df):
    data_list = []
    for _, row in df.iterrows():
        data_list.append({
            "name": row['name'],
            "coordinates": [[p[1], p[0]] for p in row['geometry'].exterior.coords],  # Convert to [longitude, latitude]
            "elevation": row['elevation']
        })
    return data_list

# Convert the GeoDataFrame to a list of dictionaries (JSON format)
locations_data = geodataframe_to_json(roof_df)

# Prepare the data to match the API's expected format
request_data = {"roofs": locations_data}

# The URL of the FastAPI endpoint
url = "http://127.0.0.1:8000/calculate_solar_potential_batch/"

# Send the POST request to the API
response = requests.post(url, json=request_data)

# Check the response
if response.status_code == 200:
    print("Response from API:")
    print(json.dumps(response.json(), indent=2))
else:
    print(f"Error: {response.status_code}")
    print(response.text)


Error: 500
Internal Server Error
